In [17]:
import json
import numpy as np
import matplotlib.pyplot as plt

NANOS_PER_SEC = 1_000_000_000

# Splitting Heuristics

## Uniformly distributed spheres

First we want to get a general impression of how the BVH will behave when performing ray intersection querries depending on which heuristic was used to construct it. To this end spheres with an equal radius R, are placed uniformly in a unit kube centered around (0, 0, -1). The radius is chosen so that the average volume filled with spheres remains constant, this is done to prevent spheres from overlapping too much.

Since the scene is uniform, I expect that all heuristics will be quite close to eachother, with the SAH leading slightly because it has a superior stopping criteria based on a cost metric. Object median split's and space median split's stopping criteria depends only on the amount of objects left or the inability to partition the remaining objects.

In an ideal case where no bounding boxes overlap and the BVH is a full tree the amount of intersection tests can be calculated as nb_pixels * 2 * log2(N), the 2 comes from the fact that at each level 2 bounbing boxes have to be tested.

In [ ]:
with open("../experiments/results/splitting_heuristics_equal_spheres_uniform_position.json") as f:
    data = json.load(f)
    
    nb_spheres = np.array(data['nb_objects'])
    
    fig, axs = plt.subplots(1, 2, sharey=True, figsize=(15, 7))
#     fig.suptitle("Uniformly distributed spheres with an equal radius")

        
    for (splitting_heuristic, intersection_tests) in sorted(data['results'].items()):        
        results = [np.array(k) for k in intersection_tests]
        averages = np.array([np.average(res) for res in results])
        stds = np.array([np.std(res) for res in results])
        
        if "Alternate" in splitting_heuristic:
            axs[0].errorbar(nb_spheres, averages, yerr=stds, fmt='-o', label=splitting_heuristic.split(" ")[1])
        else:
            axs[1].errorbar(nb_spheres, averages, yerr=stds, fmt='-o', label=splitting_heuristic.split(" ")[1])
       
    axs[0].set(ylabel="intersection tests", title="Alternating axis")
    axs[1].set(title="Longest axis")
    
    for ax in axs:    
        ideal = 640 * 640 * 2 * np.log2(nb_spheres)
        ax.plot(nb_spheres, ideal, label="ideal")
        
        ax.grid(True)
        ax.set_yscale("linear")
        ax.set_xscale("log")
        ax.set(xlabel="spheres")
        ax.legend()
        plt.savefig("../report/plots/splitting_heuristisc_equal_spheres_uniform_position.png")

The graph does show indeed that all splitting heuristics constructs BVHs that are quite similar to eachother in terms of the amount of intersection tests needed to render the scene, with the SAH slightly better. Another thing worth noting that was quite unexpected is that the standard deviation of object median split is so high.

This graph also shows that on average the required intersection tests is still an order of magnitude greate than what would be expected in an ideal scenario, this can be explained by the fact that in this experiment bounding boxes will overlap.

When adding the amount of intersection tests that would be required by a naive approach to the graphs, it can be seen that this difference in negligible compared to the speedup from use a BVH.

In [ ]:
with open("../experiments/results/splitting_heuristics_equal_spheres_uniform_position.json") as f:
    data = json.load(f)
    
    nb_spheres = np.array(data['nb_objects'])
    
    fig, axs = plt.subplots(1, 2, sharey=True, figsize=(15, 7))
#     fig.suptitle("Uniformly distributed spheres with an equal radius")

        
    for (splitting_heuristic, intersection_tests) in sorted(data['results'].items()):        
        results = [np.array(k) for k in intersection_tests]
        averages = np.array([np.average(res) for res in results])
        stds = np.array([np.std(res) for res in results])
        
        if "Alternate" in splitting_heuristic:
            axs[0].errorbar(nb_spheres, averages, yerr=stds, fmt='-o', label=splitting_heuristic.split(" ")[1])
        else:
            axs[1].errorbar(nb_spheres, averages, yerr=stds, fmt='-o', label=splitting_heuristic.split(" ")[1])
       
    axs[0].set(ylabel="intersection tests", title="Alternating axis")
    axs[1].set(title="Longest axis")
    
    for ax in axs:    
        ideal = 640 * 640 * 2 * np.log2(nb_spheres)
        ax.plot(nb_spheres, ideal, label="ideal")
        
        naive = 640 * 640 * nb_spheres
        ax.plot(nb_spheres, naive, label="naieve")
        
        ax.grid(True)
        ax.set_yscale("linear")
        ax.set_xscale("linear")
        ax.set(xlabel="spheres")
        ax.legend()
        plt.savefig("../report/plots/splitting_heuristisc_equal_spheres_uniform_position_with_naive.png")

The amount of internal nodes is O(N)

## Time complexities

### SAH

In the best case the shapes are split into 2 equal parts with each partitioning:

n + 2 n/2 + 4 n/4 + ... + log2(n) n/log2(n) = O(n log2(n))

In the worst case the nodes are partitioned in a set of length one and a set of length n-1:

n + (n-1) + (n-2) + ... + 2 = n(n + 1)/2 = O(n^2)

### Space median split

Complexity is also dominated by the partitioning, so idem SAH

### Object median split

The shapes are always split into 2 equal halves, so we get a full binary tree, at each internal node the remaining centroings have to be sorted (implemented with Pattern-defeating quicksort), which is O(n log2(n)):

n log2(n) + 2 n/2 log2(n/2)  + 4 n/4 log2(n/4) + ... + log2(n) n/log2(n) log2(n/log2(n))

= n log2(n) + [n log2(n) - 1] + [n log2(n) - 2] + ... + [n log2(n) - log2(log2(n))]

= O(n log2(n)^2)


In [ ]:
with open("../experiments/results/splitting_heuristics_equal_spheres_uniform_position_time.json") as f:
    data = json.load(f)
    
    nb_spheres = np.array(data['nb_objects'])
    
    fig, axs = plt.subplots(1, 2, sharey=False, figsize=(15, 7))
#     fig.suptitle("Uniformly distributed spheres with a uniformly distributed radius")

        
    for (splitting_heuristic, durations) in sorted(data['results'].items()):    
        results = [np.array([(d["secs"] * NANOS_PER_SEC + d["nanos"]) / 1000000.0 for d in ds]) for ds in durations]
        averages = np.array([np.average(res) for res in results])
        stds = np.array([np.std(res) for res in results])
        
        if "Alternate" in splitting_heuristic:
            axs[0].plot(nb_spheres, averages / nb_spheres, label=splitting_heuristic.split(" ")[1])
        else:
            axs[1].errorbar(nb_spheres, averages, yerr=stds, fmt='-o', label=splitting_heuristic.split(" ")[1])
       
    axs[0].set(ylabel="world build time[ms]", title="Alternating axis")
    axs[1].set(title="Longest axis")
    
    for ax in axs:    
        ax.grid(True)
        ax.set_yscale("linear")
        ax.set_xscale("log")
        ax.set(xlabel="spheres")
        ax.legend()
#         plt.savefig("../report/plots/splitting_heuristisc_uniform_spheres_uniform_position.png")

In [ ]:
with open("../experiments/results/splitting_heuristics_uniform_spheres_uniform_position.json") as f:
    data = json.load(f)
    
    nb_spheres = np.array(data['nb_objects'])
    
    fig, axs = plt.subplots(1, 2, sharey=True, figsize=(15, 7))
#     fig.suptitle("Uniformly distributed spheres with a uniformly distributed radius")

        
    for (splitting_heuristic, intersection_tests) in sorted(data['results'].items()):        
        results = [np.array(k) for k in intersection_tests]
        averages = np.array([np.average(res) for res in results])
        stds = np.array([np.std(res) for res in results])
        
        if "Alternate" in splitting_heuristic:
            axs[0].errorbar(nb_spheres, averages, yerr=stds, fmt='-o', label=splitting_heuristic.split(" ")[1])
        else:
            axs[1].errorbar(nb_spheres, averages, yerr=stds, fmt='-o', label=splitting_heuristic.split(" ")[1])
       
    axs[0].set(ylabel="intersection tests", title="Alternating axis")
    axs[1].set(title="Longest axis")
    
    for ax in axs:    
        ideal = 640 * 640 * 2 * np.log2(nb_spheres)
        ax.plot(nb_spheres, ideal, label="ideal")
        
        ax.grid(True)
        ax.set_yscale("linear")
        ax.set_xscale("linear")
        ax.set(xlabel="spheres")
        ax.legend()
        plt.savefig("../report/plots/splitting_heuristisc_uniform_spheres_uniform_position.png")

In [ ]:
with open("../experiments/results/splitting_heuristics_uniform_spheres_uniform_position_time.json") as f:
    data = json.load(f)
    
    nb_spheres = np.array(data['nb_objects'])
    
    fig, axs = plt.subplots(1, 2, sharey=True, figsize=(15, 7))
#     fig.suptitle("Uniformly distributed spheres with a uniformly distributed radius")

        
    for (splitting_heuristic, durations) in sorted(data['results'].items()):
        results = [np.array([(d["secs"] * NANOS_PER_SEC + d["nanos"]) / 1000000.0 for d in ds]) for ds in durations]
        averages = np.array([np.average(res) for res in results])
        stds = np.array([np.std(res) for res in results])
        
        if "Alternate" in splitting_heuristic:
            axs[0].errorbar(nb_spheres, averages, yerr=stds, fmt='-o', label=splitting_heuristic.split(" ")[1])
        else:
            axs[1].errorbar(nb_spheres, averages, yerr=stds, fmt='-o', label=splitting_heuristic.split(" ")[1])
       
    axs[0].set(ylabel="world build time[ms]", title="Alternating axis")
    axs[1].set(title="Longest axis")
    
    for ax in axs:
        
        ax.grid(True)
        ax.set_yscale("linear")
        ax.set_xscale("linear")
        ax.set(xlabel="spheres")
        ax.legend()
#         plt.savefig("../report/plots/splitting_heuristisc_uniform_spheres_uniform_position.png")

## Non-uniformly distributed spheres

In [ ]:
with open("../experiments/results/splitting_heuristics_equal_spheres_normal_yz.json") as f:
    data = json.load(f)
    
    nb_spheres = np.array(data['nb_objects'])
    
    fig, axs = plt.subplots(1, 2, sharey=True, figsize=(15, 7))
#     fig.suptitle("Normal distributed spheres along y, z and uniform along x with an equal radius")

        
    for (splitting_heuristic, intersection_tests) in sorted(data['results'].items()):        
        results = [np.array(k) for k in intersection_tests]
        averages = np.array([np.average(res) for res in results])
        stds = np.array([np.std(res) for res in results])
        
        if "Alternate" in splitting_heuristic:
            axs[0].errorbar(nb_spheres, averages, yerr=stds, fmt='-o', label=splitting_heuristic.split(" ")[1])
        else:
            axs[1].errorbar(nb_spheres, averages, yerr=stds, fmt='-o', label=splitting_heuristic.split(" ")[1])
       
    axs[0].set(ylabel="intersection tests", title="Alternating axis")
    axs[1].set(title="Longest axis")
    
    for ax in axs:    
        ideal = 640 * 640 * 2 * np.log2(nb_spheres)
        ax.plot(nb_spheres, ideal, label="ideal")
        
        ax.grid(True)
        ax.set_yscale("linear")
        ax.set_xscale("linear")
        ax.set(xlabel="spheres")
        ax.legend()
        plt.savefig("../report/plots/splitting_heuristics_equal_spheres_normal_yz.png")

In [ ]:
with open("../experiments/results/splitting_heuristics_equal_spheres_normal_yz_time.json") as f:
    data = json.load(f)
    
    nb_spheres = np.array(data['nb_objects'])
    
    fig, axs = plt.subplots(1, 2, sharey=True, figsize=(15, 7))
#     fig.suptitle("Uniformly distributed spheres with a uniformly distributed radius")

        
    for (splitting_heuristic, durations) in sorted(data['results'].items()):    
        results = [np.array([(d["secs"] * NANOS_PER_SEC + d["nanos"]) / 1000000.0 for d in ds]) for ds in durations]
        averages = np.array([np.average(res) for res in results])
        stds = np.array([np.std(res) for res in results])
        
        if "Alternate" in splitting_heuristic:
            axs[0].errorbar(nb_spheres, averages, yerr=stds, fmt='-o', label=splitting_heuristic.split(" ")[1])
        else:
            axs[1].errorbar(nb_spheres, averages, yerr=stds, fmt='-o', label=splitting_heuristic.split(" ")[1])
       
    axs[0].set(ylabel="world build time[ms]", title="Alternating axis")
    axs[1].set(title="Longest axis")
    
    for ax in axs:
        
        ax.grid(True)
        ax.set_yscale("linear")
        ax.set_xscale("linear")
        ax.set(xlabel="spheres")
        ax.legend()
#         plt.savefig("../report/plots/splitting_heuristisc_uniform_spheres_uniform_position.png")

In [ ]:
with open("../experiments/results/splitting_heuristics_equal_spheres_uniform_yz.json") as f:
    data = json.load(f)
    
    nb_spheres = np.array(data['nb_objects'])
    
    fig, axs = plt.subplots(1, 2, sharey=True, figsize=(15, 7))
#     fig.suptitle("Normal distributed spheres along y, z and uniform along x with an equal radius")

        
    for (splitting_heuristic, intersection_tests) in sorted(data['results'].items()):        
        results = [np.array(k) for k in intersection_tests]
        averages = np.array([np.average(res) for res in results])
        stds = np.array([np.std(res) for res in results])
        
        if "Alternate" in splitting_heuristic:
            axs[0].errorbar(nb_spheres, averages, yerr=stds, fmt='-o', label=splitting_heuristic.split(" ")[1])
        else:
            axs[1].errorbar(nb_spheres, averages, yerr=stds, fmt='-o', label=splitting_heuristic.split(" ")[1])
       
    axs[0].set(ylabel="intersection tests", title="Alternating axis")
    axs[1].set(title="Longest axis")
    
    for ax in axs:    
        ideal = 640 * 640 * 2 * np.log2(nb_spheres)
        ax.plot(nb_spheres, ideal, label="ideal")
        
        ax.grid(True)
        ax.set_yscale("linear")
        ax.set_xscale("linear")
        ax.set(xlabel="spheres")
        ax.legend()
        plt.savefig("../report/plots/splitting_heuristics_equal_spheres_uniform_yz.png")

In [ ]:
with open("../experiments/results/splitting_heuristics_equal_spheres_uniform_yz_time.json") as f:
    data = json.load(f)
    
    nb_spheres = np.array(data['nb_objects'])
    
    fig, axs = plt.subplots(1, 2, sharey=True, figsize=(15, 7))
#     fig.suptitle("Uniformly distributed spheres with a uniformly distributed radius")

        
    for (splitting_heuristic, durations) in sorted(data['results'].items()):    
        results = [np.array([(d["secs"] * NANOS_PER_SEC + d["nanos"]) / 1000000.0 for d in ds]) for ds in durations]
        averages = np.array([np.average(res) for res in results])
        stds = np.array([np.std(res) for res in results])
        
        if "Alternate" in splitting_heuristic:
            axs[0].errorbar(nb_spheres, averages, yerr=stds, fmt='-o', label=splitting_heuristic.split(" ")[1])
        else:
            axs[1].errorbar(nb_spheres, averages, yerr=stds, fmt='-o', label=splitting_heuristic.split(" ")[1])
       
    axs[0].set(ylabel="world build time[ms]", title="Alternating axis")
    axs[1].set(title="Longest axis")
    
    for ax in axs:
        
        ax.grid(True)
        ax.set_yscale("linear")
        ax.set_xscale("linear")
        ax.set(xlabel="spheres")
        ax.legend()
#         plt.savefig("../report/plots/splitting_heuristisc_uniform_spheres_uniform_position.png")

In [ ]:
with open("../experiments/results/splitting_heuristics_equal_spheres_beta_corners.json") as f:
    data = json.load(f)
    
    nb_spheres = np.array(data['nb_objects'])
    
    fig, axs = plt.subplots(1, 2, sharey=True, figsize=(15, 7))
#     fig.suptitle("Spheres in corners with an equal radius")

        
    for (splitting_heuristic, intersection_tests) in sorted(data['results'].items()):        
        results = [np.array(k) for k in intersection_tests]
        averages = np.array([np.average(res) for res in results])
        stds = np.array([np.std(res) for res in results])
        
        if "Alternate" in splitting_heuristic:
            axs[0].errorbar(nb_spheres, averages, yerr=stds, fmt='-o', label=splitting_heuristic.split(" ")[1])
        else:
            axs[1].errorbar(nb_spheres, averages, yerr=stds, fmt='-o', label=splitting_heuristic.split(" ")[1])
       
    axs[0].set(ylabel="intersection tests", title="Alternating axis")
    axs[1].set(title="Longest axis")
    
    for ax in axs:    
        ideal = 640 * 640 * 2 * np.log2(nb_spheres)
        ax.plot(nb_spheres, ideal, label="ideal")
        
        ax.grid(True)
        ax.set_yscale("linear")
        ax.set_xscale("linear")
        ax.set(xlabel="spheres")
        ax.legend()
        plt.savefig("../report/plots/splitting_heuristics_equal_spheres_beta_corners.png")

In [ ]:
with open("../experiments/results/splitting_heuristics_equal_spheres_beta_corners_time.json") as f:
    data = json.load(f)
    
    nb_spheres = np.array(data['nb_objects'])
    
    fig, axs = plt.subplots(1, 2, sharey=True, figsize=(15, 7))
#     fig.suptitle("Uniformly distributed spheres with a uniformly distributed radius")

        
    for (splitting_heuristic, durations) in sorted(data['results'].items()):    
        results = [np.array([(d["secs"] * NANOS_PER_SEC + d["nanos"]) / 1000000.0 for d in ds]) for ds in durations]
        averages = np.array([np.average(res) for res in results])
        stds = np.array([np.std(res) for res in results])
        
        if "Alternate" in splitting_heuristic:
            axs[0].errorbar(nb_spheres, averages, yerr=stds, fmt='-o', label=splitting_heuristic.split(" ")[1])
        else:
            axs[1].errorbar(nb_spheres, averages, yerr=stds, fmt='-o', label=splitting_heuristic.split(" ")[1])
       
    axs[0].set(ylabel="world build time[ms]", title="Alternating axis")
    axs[1].set(title="Longest axis")
    
    for ax in axs:
        
        ax.grid(True)
        ax.set_yscale("linear")
        ax.set_xscale("linear")
        ax.set(xlabel="spheres")
        ax.legend()
#         plt.savefig("../report/plots/splitting_heuristisc_uniform_spheres_uniform_position.png")

In [ ]:
with open("../experiments/results/splitting_heuristics_equal_spheres_beta_corners2.json") as f:
    data = json.load(f)
    
    nb_spheres = np.array(data['nb_objects'])
    
    fig, axs = plt.subplots(1, 2, sharey=True, figsize=(15, 7))
    fig.suptitle("Spheres in corners with an equal radius")

        
    for (splitting_heuristic, intersection_tests) in sorted(data['results'].items()):        
        results = [np.array(k) for k in intersection_tests]
        averages = np.array([np.average(res) for res in results])
        stds = np.array([np.std(res) for res in results])
        
        if "Alternate" in splitting_heuristic:
            axs[0].errorbar(nb_spheres, averages, yerr=stds, fmt='-o', label=splitting_heuristic.split(" ")[1])
        else:
            axs[1].errorbar(nb_spheres, averages, yerr=stds, fmt='-o', label=splitting_heuristic.split(" ")[1])
       
    axs[0].set(ylabel="intersection tests", title="Alternating axis")
    axs[1].set(title="Longest axis")
    
    for ax in axs:    
        ideal = 640 * 640 * 2 * np.log2(nb_spheres)
        ax.plot(nb_spheres, ideal, label="ideal")
        
        ax.grid(True)
        ax.set_yscale("linear")
        ax.set_xscale("linear")
        ax.set(xlabel="spheres")
        ax.legend()
        # plt.savefig('../experiments/results/plots/compare_splitting_heuristics2.png')

In [ ]:
with open("../experiments/results/splitting_heuristics_equal_spheres_beta_x.json") as f:
    data = json.load(f)
    
    nb_spheres = np.array(data['nb_objects'])
    
    fig, axs = plt.subplots(1, 2, sharey=True, figsize=(15, 7))
    fig.suptitle("Uniformly distributed spheres with an equal radius")

        
    for (splitting_heuristic, intersection_tests) in sorted(data['results'].items()):        
        results = [np.array(k) for k in intersection_tests]
        averages = np.array([np.average(res) for res in results])
        stds = np.array([np.std(res) for res in results])
        
        if "Alternate" in splitting_heuristic:
            axs[0].errorbar(nb_spheres, averages, yerr=stds, fmt='-o', label=splitting_heuristic.split(" ")[1])
        else:
            axs[1].errorbar(nb_spheres, averages, yerr=stds, fmt='-o', label=splitting_heuristic.split(" ")[1])
       
    axs[0].set(ylabel="intersection tests", title="Alternating axis")
    axs[1].set(title="Longest axis")
    
    for ax in axs:    
        ideal = 640 * 640 * 2 * np.log2(nb_spheres)
        ax.plot(nb_spheres, ideal, label="ideal")
        
        ax.grid(True)
        ax.set_yscale("linear")
        ax.set_xscale("linear")
        ax.set(xlabel="spheres")
        ax.legend()
        #     plt.savefig('../experiments/results/plots/compare_splitting_heuristics2.png')

In [ ]:
fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(15, 7))
    
with open("../renders/intersection_tests_sah.txt") as file:
    counts = list(map(int, file.read().split(",")))
    counts = [counts[i:i+640] for i in range(0, len(counts), 640)]
    axs[0].imshow(counts, origin="lower", interpolation="none")
    axs[0].get_xaxis().set_visible(False)
    axs[0].get_yaxis().set_visible(False)
    axs[0].set(title="Surface area heuristic")

with open("../renders/intersection_tests_space_median.txt") as file:
    counts = list(map(int, file.read().split(",")))
    counts = [counts[i:i+640] for i in range(0, len(counts), 640)]
    pcm = axs[1].imshow(counts, origin="lower", interpolation="none")
    axs[1].get_xaxis().set_visible(False)
    axs[1].get_yaxis().set_visible(False)
    axs[1].set(title="Space median split")
    
    fig.colorbar(pcm, ax=axs, shrink=0.75)
    
# plt.savefig("../experiments/results/plots/splitting_heuristics_equal_spheres_beta_corners_false_color.png")

# Object instancing

## Rendering performance

In [ ]:
with open("../experiments/results/instanced_intersections.json") as f1, open("../experiments/results/flattened_intersections.json") as f2:
    instanced_data = json.load(f1)
    flattened_data = json.load(f2)
    
    nb_objects = np.array(instanced_data["nb_objects"])
    
    fig, ax = plt.subplots()
        
    results = [np.array(k) for k in instanced_data["results"]["Alternate, SurfaceAreaHeuristic(12)"]]
    averages = np.array([np.average(res) for res in results])
    stds = np.array([np.std(res) for res in results])
    ax.errorbar(nb_objects, averages, yerr=stds, fmt="-o", label="instancing")
    
    results = [np.array(k) for k in flattened_data["results"]["Alternate, SurfaceAreaHeuristic(12)"]]
    averages = np.array([np.average(res) for res in results])
    stds = np.array([np.std(res) for res in results])
    ax.errorbar(nb_objects, averages, yerr=stds, fmt="-o", label="triangle soup")
        
    ax.set_yscale("linear")
    ax.set_xscale("linear")
    ax.set(xlabel="bunnies", ylabel="intersection tests")
    ax.grid(True)
    ax.legend()
    plt.savefig("../experiments/results/plots/instancing_intersection_tests.png")

As we can see from the graph a triangle soup requires slightly less intersection tests when rendering the image. An experiment with significantly more bunnies is required to verify if this gap continues to widen. While performing the experiments for this part I noticed that the time it took to construct the BVH increased significantly, up to the point that it seemed to take longer than the actual rendering. This could be a subject for further testing if I had more time available.

## Memory

This experiment uses [jemalloc](http://jemalloc.net) to heap memory required to store the world. The amount of allocated heap memory was measured before creating the world and after the world has been created, the difference is then the amount of heap memory used by the world.

Because instancing requires very few additional data, just the tranformation matrixes and a pointer to the mesh (which is negligible in comparison to the mesh), I expect that the memory usage will remain relatively constant. Without instancing I expect the memory usage to increase linearly with the amount of "instances" in the scene

In [ ]:
with open("../experiments/results/instanced_memory.json") as f1, open("../experiments/results/flattened_memory.json") as f2:
    instanced_data = json.load(f1)
    flattened_data = json.load(f2)
    
    nb_objects = np.array(instanced_data["nb_objects"])
    
    fig, ax = plt.subplots()
    ax.set_yscale("linear")
    ax.set_xscale("linear")
    ax.set(xlabel="bunnies", ylabel="heap memory[MB]")
        
    results = [np.array(k) for k in instanced_data["results"]["Alternate, SurfaceAreaHeuristic(12)"]]
    averages = np.array([np.average(res) for res in results]) / 1000000
    stds = np.array([np.std(res) for res in results]) / 1000000
    ax.errorbar(nb_objects, averages, yerr=stds, fmt="-o", label="instancing")
    
    results = [np.array(k) for k in flattened_data["results"]["Alternate, SurfaceAreaHeuristic(12)"]]
    averages = np.array([np.average(res) for res in results]) / 1000000
    stds = np.array([np.std(res) for res in results]) / 1000000
    ax.errorbar(nb_objects, averages, yerr=stds, fmt="-o", label="triangle soup")
        
    ax.grid(True)
    ax.legend()
    plt.savefig("../experiments/results/plots/instancing_heap_memory.png")

Besides the high BVH construction times and the very high memory usage, there is another downside to not using instancing, although this one can be quite subjective. With the use of instancing you can easily compose a scene programatically from smaller components, this proved to be quite cumbersome in my implementation (this approach was used to set up the experiment), another way to resolve this issue could be to by introducing an additional translation step that flattens a scene graph. This last approach has the added drawback that almost all data structure used in the scene graph have to be cloneable (implement the Clone trait in Rust, this won't be covered further here), which again wasn't the case in my implementation as it used instancing from the start.

Because of all these benefits instancing is a sane default. For very large scenes that we want to render from multiple camera perspectives, flattening the scene graph might be beneficial if the the speedup outwheighs the increased BVH construction time.